## Housing project_Regression
 

## 1. Regression Preprocessing 

In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np
import pandas as pd

# Import data

path = r"C:\Users\Aida\OneDrive\Documents\Bootcamp_WBS\Primer\Python\WBS_DATA\8_SUP_ML\Regression\Data\housing_iteration_6_regression.csv"
data = pd.read_csv(path)

# Separate target feature and predictor features
X = data.drop(columns=['SalePrice']) # X: All columns except 'SalePrice' (features)
y = data['SalePrice']  # y: The 'SalePrice' column (target)

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31416)




In [36]:
# Build preprocessor
X_cat = X_train.select_dtypes(exclude="number").copy()
X_num = X_train.select_dtypes(include="number").copy()

# We have to take extra steps to maintain feature names if we want to know their coefficients
# Encoders and ColumnTransformers will remove feature names unless .set_output is specified

# Step 1: Defining ordinal & onehot columns
ordinal_cols = [
    'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'KitchenQual', 'FireplaceQu', 
    'PoolQC', 'HeatingQC', 'GarageFinish', 'GarageQual',
    'GarageCond', 'PavedDrive','LandSlope', 'LotShape', 'OverallCond', 'OverallQual'
]

onehot_cols = [
    'MSSubClass', 'MSZoning', 'Street', 'Alley', 'LandContour', 'Utilities', 'LotConfig', 
    'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 
    'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 
    'CentralAir', 'Electrical', 'GarageType', 'Fence', 'SaleType', 'SaleCondition'
]

# Step 2: Defining the categorical encoder (with "N_A" for missing values)
# Corrected ordinal_categories
ordinal_categories = [
    ['Ex', 'Gd', 'TA', 'Fa', 'Po'],  # ExterQual
    ['Ex', 'Gd', 'TA', 'Fa', 'Po'],  # ExterCond
    ['Ex', 'Gd', 'TA', 'Fa', 'Po'],  # BsmtQual
    ['Ex', 'Gd', 'TA', 'Fa', 'Po'],  # BsmtCond
    ['Gd', 'Av', 'Mn', 'No'],        # BsmtExposure
    ['Ex', 'Gd', 'TA', 'Fa', 'Po'],  # KitchenQual
    ['Ex', 'Gd', 'TA', 'Fa', 'Po'],  # FireplaceQu
    ['Ex', 'Gd', 'TA', 'Fa', 'Po'],  # PoolQC
    ['Ex', 'Gd', 'TA', 'Fa', 'Po'],  # HeatingQC
    ['Fin', 'RFn', 'Unf'],           # GarageFinish
    ['Ex', 'Gd', 'TA', 'Fa', 'Po'],  # GarageQual
    ['Ex', 'Gd', 'TA', 'Fa', 'Po'],  # GarageCond
    ['Y', 'P', 'N'],                 # PavedDrive
    ['Gtl', 'Mod', 'Sev'],           # LandSlope
    ['Reg', 'IR1', 'IR2', 'IR3'],    # LotShape
    [10, 9, 8, 7, 6, 5, 4, 3, 2, 1], # OverallCond
    [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]  # OverallQual
]

In [37]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder


# Ordinal pipeline
ordinal_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="constant", fill_value="N_A")),  # Fill missing values
    ('ordinal', OrdinalEncoder(categories=ordinal_categories, handle_unknown='use_encoded_value', unknown_value=-1))  # Ordinal encoding
])

# One-hot pipeline
onehot_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="constant", fill_value="N_A")),  # Fill missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # One-hot encoding
])

# Numerical pipeline
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with mean
    ('scaler', StandardScaler())  # Standard scaling
])

# Combine pipelines with ColumnTransformer
full_preprocessing = ColumnTransformer(transformers=[
    ("num_pipe", numeric_pipeline, X_num.columns),       # Apply numerical pipeline to numerical columns
    ("ordinal", ordinal_pipeline, ordinal_cols),        # Apply ordinal pipeline to ordinal columns
    ("onehot", onehot_pipeline, onehot_cols)            # Apply one-hot pipeline to categorical columns
])

# Fit and transform the full preprocessing pipeline
X_transformed_full = full_preprocessing.fit_transform(X_train)

# Display the shape of the transformed data
print(f"Transformed dataset shape: {X_transformed_full.shape}")


Transformed dataset shape: (1168, 238)


### Predicting with the SGD REGRESSOR MODEL

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV

# === SGD REGRESSOR MODEL ===
full_pipeline = make_pipeline(
    full_preprocessing,  # Preprocessing steps
    SGDRegressor(random_state=42, max_iter=1000, tol=1e-3)  # SGD Regressor
)

# === HYPERPARAMETER GRID ===
param_grid = {
    "sgdregressor__alpha": [0.0001, 0.001, 0.01, 0.1],  # Regularization strength
    "sgdregressor__penalty": ["l2", "l1", "elasticnet"],  # Regularization type
    "sgdregressor__learning_rate": ["constant", "optimal", "invscaling", "adaptive"],  # Learning rate schedule
    "sgdregressor__eta0": [0.01, 0.1, 1],  # Initial learning rate
    "sgdregressor__max_iter": [500, 1000, 2000],  # Maximum iterations
}

# === RANDOMIZED SEARCH CV ===
search = RandomizedSearchCV(
    full_pipeline,
    param_distributions=param_grid,
    n_iter=50,  # Number of random combinations
    cv=5,  # 5-fold cross-validation
    verbose=1,
    random_state=42,
    scoring="r2",  # Optimize for R^2
    n_jobs=-1  # Use all CPU cores
)

# === FITTING THE MODEL ===
search.fit(X_train, y_train)

# === RESULTS ===
print("Best Parameters:", search.best_params_)
print("Best Score (R^2):", search.best_score_)


## 3. Error analysis

In [42]:
from sklearn.metrics import r2_score


# Make predictions on the training data
y_train_pred = search.predict(X_train)

# Calculate accuracy score
r2 = r2_score(y_train, y_train_pred )
print(f"R² Score: {r2}")

R² Score: 0.9729886579748962


In [43]:
y_test_pred = search.predict(X_test)  # Predictions on test data
r2 = r2_score(y_test, y_test_pred) 
print(f"R² Score: {r2}")   # Compare to test labels

R² Score: 0.8511378012715138
